In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
sql = '''
select tmu.id_card_num, om.order_number, om.create_time  from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
'''
df = query(sql)

In [ ]:
# 根据id_card_num分组，计算每个组的order_number数量
df.loc[:, 'num'] = df.groupby('id_card_num').order_number.transform('count')
df


,id_card_num,order_number,create_time,是否复租,num
760,11010119871229251X,A2023022214341835,2023-02-22 14:34:19,0,2
8704,11010119871229251X,A2023113020425324,2023-11-30 20:42:53,1,2
11111,110104198408231236,A2024031308293314,2024-03-13 08:29:34,0,2
37312,110104198408231236,A2025031711014312,2025-03-17 11:01:43,1,2
7629,110106198907142142,A20231110185028116,2023-11-10 18:50:29,0,2
...,...,...,...,...,...
23191,654025200307040516,A2024092016081255,2024-09-20 16:08:12,0,2
17879,654125198306234226,A2024071719121929,2024-07-17 19:12:20,0,2
45974,654125198306234226,A20250519114048520,2025-05-19 11:40:48,1,2
12659,659001199811171214,A202404171301013,2024-04-17 13:01:01,0,2


In [ ]:
# 根据id_card_num分组，计算每个组的create_time的最小值
df.loc[:, 'min_time'] = df.groupby('id_card_num').create_time.transform('min')
# 
df.loc[:, '是否复租'] = np.where(df.create_time==df.min_time, 0, 1)
df = df[df.num>1][['id_card_num', 'order_number', 'create_time', '是否复租']].sort_values(by=['id_card_num', 'create_time'])
with pd.ExcelWriter('D:/需求/总需求/复租订单.xlsx') as writer:
    df.to_excel(writer, sheet_name='复租订单', index=False)